# Major League Soccer Player & Salary 2007-2022

##### Introduction
This project analyses salary and player data for the MLS between years 2007-2022. \
PDF-files (16 files, one for each year) were converted into csv-files using tabula.\
The dataset contains the name of the "Club" of a respective player, the players "First Name", "Last Name", "Base Salary", and Guaranteed "Compensation". The respective relevant year was added as extra column in Excel, so data can be analysed according to year after merging all data into one DataFrame.

The data was obtained from https://mlsplayers.org/resources/salary-guide in pdf files, which also contains explanations on Guaranteed Compensation versus Base Salary.

This notebook produces some csv files used for Streamlit application and is only used for EDA

#### Import modules

In [ ]:
# importing the required modules
import glob
import pandas as pd
import scipy as sp
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 600)

#### Specify directory containing csv files and concat into one large df

In [ ]:
#current dir
cwd = os.getcwd()
# specifying the path to csv files
path = cwd
path

In [ ]:
# csv files in the path
file_list = glob.glob(path + "/*.csv")
#print(file_list)
 
# list of csv files to merge
# pd.read_csv(file_path) reads the 
# csv data into pandas dataframe
csv_list = []
 
for file in file_list:
    csv_list.append(pd.read_csv(file))

# concatenate all DataFrames in the list
# into a single DataFrame, returns new
# DataFrame
csv_merged = pd.concat(csv_list, ignore_index=True)
mls_data = csv_merged
mls_data.head(5)

#### Read in csv file containing club name and abbreviation sport code

In [ ]:
clubs_mls = pd.read_csv('C:/Users/aruta/Dropbox/Python/Jupyter/MLS_data/Clubs_MLS.csv', header=None)
clubs_mls.columns = ["Club", "Full Club"]
clubs_mls.head(20)

## EDA

#### pre-processing/evaluation

In [ ]:
#DF overview
mls_data.info()

In [ ]:
#check Club names for consistency and occurence
mls_data["Club"].value_counts()

In [ ]:
print(mls_data[mls_data["Club"] == "Pool"].head(5))
mls_data[mls_data["Club"] == "Major League Soccer"].head(5)

In [ ]:
#check player position (Pos) for consistency and occurence
mls_data["Pos"].value_counts()

EDA has revealed a few areas for cleanup:
* Base Salary and Compensation are of type object (contain string), and need to change to numeric values for calculations
* The Pos naming seems inconsistent across the years - sometimes positions are designated D-M, M-D, D/M etc. - these positions will be treated as same
* For some years the Club name is given as a three-letter code, in other years the Club name is provided in full - this needs to be consistent across the dataset for analysis. There are also some inconsistent spellings, and some surprising Club names, such as Pool, Major League Soccer, None, Retired - need to check what those reflect and possibly remove before analysis if sensible. 
* Further, Los Angeles has two teams in the MLS in the later years (LA Galaxy, Los Angeles FC). Los Angeles FC are in the MLS as of 2018, so entries with LA prior to that year represents LA Galaxy, who have been in the MLS much longer. The abbreviation for Los Angeles in mls_data (and in the original data) is LAGFC, which seems a mistake and should read LAFC (as in club_mls df). Similar, New York has two teams in the MLS - New York Red Bulls and New York City FC - New York City FC entered MLS in 2016, so every year before the designation NY refers to New York Red Bulls (NYR in club_mls df). Information on MLS teams for LA and NY were obtained from Wikipedia. Montreal is another example of possibly two teams. 

#### cleanup

In [ ]:
# change Base salary column to numeric
mls_data["Base Salary"] = mls_data["Base Salary"].str[:-3]
mls_data["Base Salary"] = mls_data["Base Salary"].str.replace(",", "")
mls_data["Base Salary"] = mls_data["Base Salary"].str.replace(".", "")
mls_data["Base Salary"] = mls_data["Base Salary"].str.replace("-", "")
mls_data["Base Salary"] = mls_data["Base Salary"].str.replace(" ", "")
mls_data["Base Salary"] = mls_data["Base Salary"].str.replace("$", "")
mls_data["Base Salary"] = pd.to_numeric(mls_data["Base Salary"])
# change Compensation column to numeric
mls_data["Compensation"] = mls_data["Compensation"].str[:-3]
mls_data["Compensation"] = mls_data["Compensation"].str.replace(",", "")
mls_data["Compensation"] = mls_data["Compensation"].str.replace(".", "")
mls_data["Compensation"] = mls_data["Compensation"].str.replace("-", "")
mls_data["Compensation"] = mls_data["Compensation"].str.replace(" ", "")
mls_data["Compensation"] = mls_data["Compensation"].str.replace("$", "")
mls_data["Compensation"] = pd.to_numeric(mls_data["Compensation"])
# homogenize position designation
mls_data["Pos"] = mls_data["Pos"].str.replace("F-M", "M-F")
mls_data["Pos"] = mls_data["Pos"].str.replace("M/F", "M-F")
mls_data["Pos"] = mls_data["Pos"].str.replace("M-D", "D-M")
mls_data["Pos"] = mls_data["Pos"].str.replace("D/M", "D-M")
mls_data["Pos"] = mls_data["Pos"].str.replace("D/F", "D-F")
mls_data["Pos"] = mls_data["Pos"].str.replace("F-D", "D-F")
mls_data["Pos"] = mls_data["Pos"].str.replace("MF", "M-F")
mls_data["Pos"] = mls_data["Pos"].str.replace("M/D", "D-M")
mls_data["Pos"] = mls_data["Pos"].str.replace("F/M", "M-F")
# correct some CLub names and align with club_mls abbreviation where possible
mls_data["Club"] = mls_data["Club"].str.replace("New England Revolutio", "New England Revolution")
mls_data["Club"] = mls_data["Club"].str.replace("New England Revolutionn", "New England Revolution")
mls_data["Club"] = mls_data["Club"].str.replace("POOL", "Pool")
mls_data["Club"] = mls_data["Club"].str.replace("OOL", "Pool")
mls_data["Club"] = mls_data["Club"].str.replace("DC", "DCU")
mls_data["Club"] = mls_data["Club"].str.replace("LA", "LAG")
mls_data["Club"] = mls_data["Club"].str.replace("LAGFC", "LAF")
mls_data["Club"] = mls_data["Club"].str.replace("LAFC", "LAF")
mls_data["Club"] = mls_data["Club"].str.replace("NY", "NYR")
mls_data["Club"] = mls_data["Club"].str.replace("NYRRB", "NYR")
mls_data["Club"] = mls_data["Club"].str.replace("NE", "NER")
mls_data["Club"] = mls_data["Club"].str.replace("NYRCFC", "NYC")
mls_data["Club"] = mls_data["Club"].str.replace("YCFC", "NYC")
mls_data["Club"] = mls_data["Club"].str.replace("MNUFC", "MIN")
mls_data["Club"] = mls_data["Club"].str.replace("SJ", "SJE")
mls_data["Club"] = mls_data["Club"].str.replace("POR", "POT")
mls_data["Club"] = mls_data["Club"].str.replace("TOR", "TFC")
mls_data["Club"] = mls_data["Club"].str.replace("KC", "SKC")
#club_mls remove of (USA) (CAN) strings
clubs_mls["Full Club"] = clubs_mls["Full Club"].str[:-6]

In [ ]:
# confirm Base Salary and Compensation has changed to numeric
mls_data.dtypes

In [ ]:
#confirm consistent Pos designation following cleanup
mls_data["Pos"].value_counts()

In [ ]:
#make new column and copy Club column
mls_data["Team"]=mls_data["Club"]
mls_data.head(5)

In [ ]:
# merge DFs on column Club
mls_data_clubnames = pd.merge(mls_data, clubs_mls, on='Club', how='outer')
mls_data_clubnames

In [ ]:
# remove rows wih certain entries in column Club
mls_data_clubnames = mls_data_clubnames[mls_data_clubnames["Club"] != "Pool"]
mls_data_clubnames = mls_data_clubnames[mls_data_clubnames["Club"] != "None"]
mls_data_clubnames = mls_data_clubnames[mls_data_clubnames["Club"] != "Retired"]
#mls_data_clubnames = mls_data_clubnames[mls_data_clubnames["Team"] != "Charlotte FC"]
#mls_data_clubnames = mls_data_clubnames[mls_data_clubnames["Club"] != "CHL"]
mls_data_clubnames = mls_data_clubnames[mls_data_clubnames["Club"].notna()]
mls_data_clubnames = mls_data_clubnames.drop('Full Club', 1)
# create column with unique player identification
mls_data_clubnames["uniquename"] = mls_data_clubnames["Last Name"] + '_' + mls_data_clubnames["First Name"]

In [ ]:
full_name_array = ["Charlotte FC", "Club Deportivo Chivas","Columbus Crew","New York Red Bulls","New England Revolution", \
                   "Atlanta United", "FC Dallas","Vancouver Whitecaps","Toronto FC","Real Salt Lake", \
                   "Orlando City SC","LAG Galaxy","Minnesota United", "San Jose Earthquakes", "Houston Dynamo", \
                   "Philadelphia Union", "Colorado Rapids", "Chicago Fire", "Sporting Kansas City", "Portland Timbers", \
                   "New York City FC", "Seattle Sounders FC", "DCU United", "FC Cincinnati", \
                   "New England Revolution", "Nashville SC", "Inter Miami", "Austin FC", "Major League Soccer", \
                   "Los Angeles FC", "Montreal", "Montreal Impact", "Montreal"]
abbrev_array = ["CHL", "CHV","CLB","NYR","NER","ATL","DAL", "VAN", "TFC", "RSL", "ORL", "LAG", "MIN", "SJE", "HOU", "PHI", "COL", \
               "CHI", "SKC", "POT", "NYC", "SEA", "DCU", "CIN", "NER", "NSC", "MIA", "AUS", "MLS", "LAF", "CFM", "CFM", "MTL"]
#full_name_array = clubs_mls["Full Club"].tolist()
#abbrev_array = clubs_mls["Club"].tolist()
for i in range(len(abbrev_array)):
    mask = mls_data_clubnames["Team"] == full_name_array[i]
    mls_data_clubnames.loc[mask, 'Club'] = abbrev_array[i]    
for i in range(len(abbrev_array)):
    mask = mls_data_clubnames["Club"] == abbrev_array[i]
    mls_data_clubnames.loc[mask, 'Team'] = full_name_array[i] 

In [ ]:
mls_data_clubnames = mls_data_clubnames[mls_data_clubnames['Base Salary'].notna()]
mls_data_clubnames["Club"].value_counts()

In [ ]:
mls_data_clubnames["Year"] = mls_data_clubnames["Year"].astype(int)

In [ ]:
mls_data_clubnames.to_csv('..\\mls_clean.csv', index=False)

After cleanup, the Club and Team names are consistent across the years, as are the designations of player positions (Pos column). Finally, non-useful entries were removed (such as retired players).

## Analysis

### Highest and lowest paid player across the years 2007-2022

#### Focus on highest paid players
* Identify highest paid players 
* Are distinct playing positions associated with highest pay?

In [ ]:
idx = mls_data_clubnames.groupby(['Year'])['Compensation'].transform(max) == mls_data_clubnames['Compensation']
highest_paid_player_each_year = mls_data_clubnames[idx].sort_values(by=['Year'])
highest_paid_player_each_year.reset_index()
highest_paid_player_each_year

In [ ]:
#grid = sns.FacetGrid(highest_paid_players, col = "Year", hue = "Last Name", col_wrap=4)
# plt.figure(figsize=(6, 6))
# sns.barplot(y = highest_paid_player_each_year["Last Name"], x = highest_paid_player_each_year["Compensation"], hue = highest_paid_player_each_year['Year'], orient='h')
# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

# idx = mls_data_clubnames.groupby(['Year'])['Compensation'].transform(max) == mls_data_clubnames['Compensation']
# highest_paid_player_each_year = mls_data_clubnames[idx].sort_values(by=['Year'])
# #plt.figure(figsize=(6, 6))
# x=sns.barplot(x = highest_paid_player_each_year["Year"].astype(int), y = highest_paid_player_each_year["Compensation"], hue = highest_paid_player_each_year['Last Name'], dodge=False)
# plt.ylabel("Compensation in million $")
# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# plt.title("Highest paid players per year")
# plt.xticks(rotation=45)
# plt.show()

In [ ]:
#plt.figure(figsize=(6, 6))
sns.barplot(x = highest_paid_player_each_year["Year"].astype(int), y = highest_paid_player_each_year["Compensation"], hue = highest_paid_player_each_year['Last Name'], dodge=False)
plt.ylabel("Compensation in million $")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.title("Highest paid players per year")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# check how many times a highest oaid player has been top earner in MLS
highest_paid_player_each_year["Last Name"].value_counts()

In [ ]:
# check which team has had highest paid players
highest_paid_player_each_year["Team"].value_counts()

In [ ]:
# check positions of highest paid players
highest_paid_player_each_year["Pos"].value_counts()

Intermediate observations on highest paid players:
* It seems that highest paid players are largely foreigners that have played professional football in leagues considered stronger than MLS.
* Also, it seems that midfielders or forwards are highest paid players, whereas defenders or goalkeepers are not represented. This needs further evaluation.

In [ ]:
#ratio of players in MLS between 2007-2022
#rppay - ratio player position all years
rppay = mls_data_clubnames["Pos"].value_counts().tolist()
rppay_list = []
for i in range(len(rppay)):
    rppay_list.append(float('%.4f' %(rppay[i]/mls_data_clubnames["Pos"].value_counts().sum())))
print(rppay_list)
pos_list_mls = mls_data_clubnames["Pos"].value_counts().reset_index()["index"].tolist()
print(pos_list_mls)

In [ ]:
#rhpppay - ratio highest paid player position all years
rhpppay = highest_paid_player_each_year["Pos"].value_counts().tolist()
rhpppay_list = []
for i in range(len(rhpppay)):
    rhpppay_list.append(float('%.4f' %(rhpppay[i]/highest_paid_player_each_year["Pos"].value_counts().sum())))
print(rhpppay_list)
pos_list_rhpppay = highest_paid_player_each_year["Pos"].value_counts().reset_index()["index"].tolist()
#pos_list_rhpppay

In [ ]:
df_pos_ratio_mls_h = pd.DataFrame(list(zip(pos_list_mls, rppay_list)),
              columns=['Pos','Pos ratio MLS'])
df_pos_ratio_rhpppay = pd.DataFrame(list(zip(pos_list_rhpppay, rhpppay_list)),
              columns=['Pos','Pos ratio highest paid'])
x = pd.merge(df_pos_ratio_mls_h, df_pos_ratio_rhpppay, on='Pos', how='outer')
# save file for streamlit
x_ = x[["Pos", "Pos ratio MLS"]]
x_.to_csv("..\\player_ratio_pos_mls.csv", index=False)
x = pd.merge(df_pos_ratio_mls_h, df_pos_ratio_rhpppay, on='Pos', how='outer').dropna()
x                                                                              

In [ ]:
idx = mls_data_clubnames.groupby(['Year'])['Compensation'].transform(min) == mls_data_clubnames['Compensation']
lowest_paid_player_each_year = mls_data_clubnames[idx].sort_values(by=['Year', 'Team'])
#lowest_paid_player_each_year

In [ ]:
#rlpppay - ratio lowest paid player position all years
rlpppay = lowest_paid_player_each_year["Pos"].value_counts().tolist()
rlpppay_list = []
for i in range(len(rlpppay)):
    rlpppay_list.append(float('%.4f' %(rlpppay[i]/lowest_paid_player_each_year["Pos"].value_counts().sum())))
print(rlpppay_list)
pos_list_rlpppay = lowest_paid_player_each_year["Pos"].value_counts().reset_index()["index"].tolist()
pos_list_rlpppay

In [ ]:
df_pos_ratio_mls_l = pd.DataFrame(list(zip(pos_list_mls, rppay_list)),
              columns=['Pos','Pos ratio MLS'])
df_pos_ratio_rlpppay = pd.DataFrame(list(zip(pos_list_rlpppay, rlpppay_list)),
              columns=['Pos','Pos ratio lowest paid'])
y = pd.merge(df_pos_ratio_mls_l, df_pos_ratio_rlpppay, on='Pos', how='outer').dropna()

In [ ]:
# Chi-Square GOF - number of expected values likely too small for Chi_square GOF test
# but will still evaluate
obs_exp_chi_squ_h = pd.merge(x, y, on='Pos', how='outer')
obs_exp_chi_squ_h['Expected_value'] = obs_exp_chi_squ_h['Pos ratio MLS_x']*highest_paid_player_each_year["Pos"].value_counts().sum()
obs_exp_chi_squ_h['Observed_value'] = obs_exp_chi_squ_h['Pos ratio highest paid']*highest_paid_player_each_year["Pos"].value_counts().sum()
obs_exp_chi_squ_h['O-E_square/expected'] = ((obs_exp_chi_squ_h['Observed_value'] - obs_exp_chi_squ_h['Expected_value'])**2)/obs_exp_chi_squ_h['Expected_value']
obs_exp_chi_squ_h['O-E_square/expected'] = obs_exp_chi_squ_h['O-E_square/expected'].round(2)
obs_exp_chi_squ_h = obs_exp_chi_squ_h.drop(["Pos ratio lowest paid", "Pos ratio MLS_y"], axis=1)
# check if statistic can be evaluated, or too many expected value below threshold
#show ob, exp value table
print(obs_exp_chi_squ_h)
print(sp.stats.chisquare(f_obs=obs_exp_chi_squ_h.Observed_value, f_exp=obs_exp_chi_squ_h.Expected_value))

In [ ]:
obs_exp_chi_squ_h

As indicated by "O-E_square/expected" for Pos M (4.96) and D (2.47), midfielders and forwards seem to be over-represented among the highest paid players. However, due to the small sample size (only 16), Chi-square GOF can't be assessed (Power_divergenceResult(statistic=nan, pvalue=nan).

#### Focus on top ten paid players each year
The analysis of a potential association of highest salary with a players position needs more data points to check statistics. Instead of using only the highest paid player for each year, we'll use the top ten paid players for each year

In [ ]:
top_ten_paid_each_year = mls_data_clubnames.sort_values(['Year','Compensation'],ascending = False).groupby('Year').head(10) 

In [ ]:
tt_year = 2022
top_ten_paid_each_year_year = top_ten_paid_each_year[top_ten_paid_each_year["Year"] == tt_year]
#plt.figure(figsize=(6, 6))
sns.barplot(x = top_ten_paid_each_year_year["Last Name"], y = top_ten_paid_each_year_year["Compensation"], hue= top_ten_paid_each_year_year["Club"], dodge=False)
plt.ylabel("Compensation in million $")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.title(f'Top ten paid players in year {tt_year}')
plt.xticks(rotation=60)
plt.show()

In [ ]:
top_ten_paid_each_year["Pos"].value_counts()

In [ ]:
#rtthpppay - ratio top ten highest paid player position all years
rtthpppay = top_ten_paid_each_year["Pos"].value_counts().tolist()
rtthpppay_list = []
for i in range(len(rtthpppay)):
    rtthpppay_list.append(float('%.4f' %(rtthpppay[i]/top_ten_paid_each_year["Pos"].value_counts().sum())))
print(rtthpppay_list)
pos_list_rtthpppay = top_ten_paid_each_year["Pos"].value_counts().reset_index()["index"].tolist()
pos_list_rtthpppay

In [ ]:
df_pos_ratio_mls_tt = pd.DataFrame(list(zip(pos_list_mls, rppay_list)),
              columns=['Pos','Pos ratio MLS'])
df_pos_ratio_rtthpppay = pd.DataFrame(list(zip(pos_list_rtthpppay, rtthpppay_list)),
              columns=['Pos','Pos ratio top ten paid'])
#z = pd.merge(df_pos_ratio_mls_tt, df_pos_ratio_rtthpppay, on='Pos', how='outer').dropna()
z = pd.merge(df_pos_ratio_mls_tt, df_pos_ratio_rtthpppay, on='Pos', how='outer')
z["Pos ratio top ten paid"] = z["Pos ratio top ten paid"].fillna(0)
z

In [ ]:
obs_exp_chi_squ_tt = z
obs_exp_chi_squ_tt['Expected_value'] = obs_exp_chi_squ_tt['Pos ratio MLS']*top_ten_paid_each_year["Pos"].value_counts().sum()
obs_exp_chi_squ_tt['Observed_value'] = obs_exp_chi_squ_tt['Pos ratio top ten paid']*top_ten_paid_each_year["Pos"].value_counts().sum()
obs_exp_chi_squ_tt['O-E_square/expected'] = ((obs_exp_chi_squ_tt['Observed_value'] - obs_exp_chi_squ_tt['Expected_value'])**2)/obs_exp_chi_squ_tt['Expected_value']
obs_exp_chi_squ_tt['O-E_square/expected'] = obs_exp_chi_squ_tt['O-E_square/expected']
obs_exp_chi_squ_tt = obs_exp_chi_squ_tt.drop(["Pos ratio top ten paid", "Pos ratio MLS"], axis=1)
# only keep M,F,D,GK for Chi-square
# obs_exp_chi_squ_tt = obs_exp_chi_squ_tt.iloc[:-3]
obs_exp_chi_squ_tt.to_csv('..\\top_ten_paid_obs_exp.csv', index=False)
#show ob, exp value table
print(obs_exp_chi_squ_tt)
#Chi-square GOF stats
print(sp.stats.chisquare(f_obs=obs_exp_chi_squ_tt.Observed_value, f_exp=obs_exp_chi_squ_tt.Expected_value))

In [ ]:
pvalue = sp.stats.chisquare(f_obs=obs_exp_chi_squ_tt.Observed_value, f_exp=obs_exp_chi_squ_tt.Expected_value)[1]
print('p-value ' + '{:.3g}'.format(pvalue))

In [ ]:
pie, ax = plt.subplots(figsize=[10,6])
labels = obs_exp_chi_squ_tt["Pos"]
plt.pie(x=obs_exp_chi_squ_tt["Expected_value"], autopct="%.1f%%", explode=[0.03]*len(obs_exp_chi_squ_tt["Observed_value"]), labels=labels, pctdistance=0.5, colors=["#CDAD00", "#5CACEE", "#9FB6CD", "#F8F8FF", "#FF7D40"])
plt.title("Distribution of playing position of players in MLS 2007-2022", fontsize=14);
plt.show()

pie, ax = plt.subplots(figsize=[10,6])
labels = obs_exp_chi_squ_tt["Pos"]
plt.pie(x=obs_exp_chi_squ_tt["Observed_value"], autopct="%.1f%%", explode=[0.03]*len(obs_exp_chi_squ_tt["Observed_value"]), labels=labels, pctdistance=0.5, colors=["#CDAD00", "#5CACEE", "#9FB6CD", "#F8F8FF", "#FF7D40"])
plt.title("Distribution of playing positions of top ten highest paid players 2007-2022", fontsize=14);

#### Focus on lowest paid players
* Identify lowest paid players 
* Are distinct playing positions associated with lowest pay?

In [ ]:
lowest_paid_player_each_year["Team"].value_counts()

In [ ]:
lowest_paid_player_each_year["Pos"].value_counts()

In [ ]:
obs_exp_chi_squ_l = pd.merge(x, y, on='Pos', how='outer')
obs_exp_chi_squ_l['Expected_value'] = obs_exp_chi_squ_l['Pos ratio MLS_y']*lowest_paid_player_each_year["Pos"].value_counts().sum()
obs_exp_chi_squ_l['Observed_value'] = obs_exp_chi_squ_l['Pos ratio lowest paid']*lowest_paid_player_each_year["Pos"].value_counts().sum()
obs_exp_chi_squ_l['O-E_square/expected'] = ((obs_exp_chi_squ_l['Observed_value'] - obs_exp_chi_squ_l['Expected_value'])**2)/obs_exp_chi_squ_l['Expected_value']
obs_exp_chi_squ_l['O-E_square/expected'] = obs_exp_chi_squ_l['O-E_square/expected'].round(2)
obs_exp_chi_squ_l = obs_exp_chi_squ_l.drop(["Pos ratio highest paid", "Pos ratio MLS_x"], axis=1)
# only keep M,F,D,GK for Chi-square
obs_exp_chi_squ_l = obs_exp_chi_squ_l.iloc[:-3]
#show ob, exp value table
print(obs_exp_chi_squ_l)
#Chi-square GOF stats
print(sp.stats.chisquare(f_obs=obs_exp_chi_squ_l.Observed_value, f_exp=obs_exp_chi_squ_l.Expected_value))

As indicated by "O-E_square/expected" for Pos F (3.20) and GK (5.63), forwards seem under-represented, and goalkeepers over-represented among the lowest paid players. Test stats as follows - Power_divergenceResult(statistic=9.755879986322011, pvalue=0.020759402200683178)

#### Visualisation of Compensation for all years allocated to Pos

In [ ]:
# display compensation of each year across player Pos
grid = sns.FacetGrid(mls_data_clubnames, col = "Year", hue = "Pos", col_wrap=4)
grid.map(sns.scatterplot, "Pos", "Compensation")

grid.add_legend()
plt.show()

In [ ]:
#estimate number of bins for histogram
bin_no = int((mls_data_clubnames["Compensation"].max()-mls_data_clubnames["Compensation"].min())/len(mls_data_clubnames))
x = mls_data_clubnames[mls_data_clubnames["Year"] == 2022]
fig = plt.figure(figsize = (10,5))
plt.hist(x.Compensation, bins=bin_no, color='skyblue', edgecolor='k', alpha=0.65)
plt.axvline(mls_data_clubnames.Compensation.mean(), color='k', linestyle='dashed', linewidth=1, label="mean compensation")
plt.axvline(mls_data_clubnames.Compensation.median(), color='b', linestyle='dashed', linewidth=1, label="median compensation")
plt.axvline(mls_data_clubnames.Compensation.max(), color='r', linestyle='dashed', linewidth=1, label="max compensation")
plt.xlabel('Compensation')
plt.ylabel('No of players')
plt.title('Compensation distribution for players in 2022')
plt.legend(loc='upper center')
plt.show()

In [ ]:
# player_all_years_sum = mls_data_clubnames.groupby("Year")["Year"]
# player_all_years_sum
# player_all_years_sum.plot(legend=True)
# plt.show()

In [ ]:
compensation_all_years_median = mls_data_clubnames.groupby("Year")["Compensation"].median().round(2)
compensation_all_years_median

In [ ]:
compensation_all_years_median.plot(legend=True)
plt.ylabel('Median compensation in $')
plt.show()

In [ ]:
compensation_all_years_sum = mls_data_clubnames.groupby("Year")["Compensation"].sum().round(2)
compensation_all_years_sum

In [ ]:
compensation_all_years_sum.plot(legend=True)
plt.ylabel('Total compensation in $ / year')
plt.show()

#### Longest serving player in MLS

In [ ]:
# create df containing players ranked regarding years in MLS
longest_serving_player_ranked = mls_data_clubnames["uniquename"].value_counts().reset_index()

# create function to display salary development for MLS players
def player_compensation_history(number_of_years = 0):
    '''This function creates a plot about compensation history of players that have served
    x-number of years in MLS.
    By default, players who have played longest in MLS will be shown.
    To view plots other than those for players who have served longest in MLS, users can provide an (int)
    argument that will be deducted from the maximum serving time.
    '''
    years_under_evaluation = longest_serving_player_ranked["uniquename"].max() - number_of_years
    print(f'Showing plot for players playing in the MLS for {years_under_evaluation} years')
    longest_serving_players_name_s = longest_serving_player_ranked[longest_serving_player_ranked["uniquename"] == longest_serving_player_ranked["uniquename"].max() - number_of_years]
    longest_serving_player_df = mls_data_clubnames[mls_data_clubnames['uniquename'].isin(longest_serving_players_name_s["index"].tolist())]
    longest_serving_player_df = longest_serving_player_df.sort_values(["Last Name", "Year"])
    # plot salary development for players serving longest in MLS
    fig_pch, ax_pch = plt.subplots(figsize=(10,4))
    for key, grp in longest_serving_player_df.groupby(['uniquename']):
        ax_pch.plot(grp['Year'], grp['Compensation'], label=key)
    ax_pch.legend()
    ax_pch.legend(bbox_to_anchor=(1.0, 1.0))
    ax_pch.set_xlabel("Year")
    ax_pch.set_ylabel("Compensation in $")
    plt.show()

In [ ]:
# checking compensation development for players with longest-, second-longest-, and
# third-longest-serving time in MLS
for i in range(3):
    player_compensation_history(i)